In [1]:
import pandas as pd
import polars as pl
import numpy as np
import re
import os
import sys
import json
import torch
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16
).to(device)

model.eval()

/scratch/arccelt/venvs/game-inn/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.06it/s]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (rotary_emb):

In [ ]:
df = pl.read_csv(
    '../data/user_reviews/raw/all_reviews/all_reviews.csv',
    columns=["recommendationid", "appid", "game", "review"],
    low_memory=True
)

In [5]:
df.schema

Schema([('recommendationid', Int64),
        ('appid', Int64),
        ('game', String),
        ('review', String)])

In [6]:
df.head(3)

recommendationid,appid,game,review
i64,i64,str,str
148919893,10,"""Counter-Strike""","""старость"""
148919350,10,"""Counter-Strike""","""Лучше кс 2"""
148913051,10,"""Counter-Strike""","""çoh iyi ama pahalı"""


In [7]:
# Count the number of reviews for each game
review_counts = (
    df
    .group_by(["appid", "game"])
    .agg(pl.len().alias("review_count"))
    .sort("review_count", descending=True)
)
review_counts.head(10)

appid,game,review_count
i64,str,u32
730,"""Counter-Strike 2""",7704653
578080,"""PUBG: BATTLEGROUNDS""",2235431
271590,"""Grand Theft Auto V""",1659263
105600,"""Terraria""",1205564
359550,"""Tom Clancy's Rainbow Six Siege""",1191091
4000,"""Garry's Mod""",1006609
440,"""Team Fortress 2""",998601
252490,"""Rust""",974388
550,"""Left 4 Dead 2""",789098


In [8]:
def make_prompt(review):
    return f"""Extract the player's perception of the game's visual and audio style from the review below.

Review: "{review}"

Return a JSON object like this:
{{
  "visual_perception": <string or null>,
  "audio_perception": <string or null>
}}

- Use English.
- If not mentioned, use JSON null (not a string).
- Output only the JSON. Nothing else.
"""

In [9]:
sample_reviews_df = df.sample(n=20, seed=42)
sample_reviews = sample_reviews_df["review"].to_list()
sample_reviews_df

recommendationid,appid,game,review
i64,i64,str,str
93432934,1328660,"""Need for Speed™ Hot Pursuit Re…","""Just a graphical upgrade with …"
16366885,218820,"""Mercenary Kings""","""ข้อดี - สนุกมาก - คราฟปืน,มีด…"
99426611,359550,"""Tom Clancy's Rainbow Six Siege""",""":) 8) me gusta bastante """
96427458,1290490,"""UNBEATABLE [white label]""","""Ohhh my!!!!! Who ever is readi…"
80204144,22490,"""Fallout: New Vegas PCR""","""9 гулей в ракете из 10"""
…,…,…,…
127110915,1369320,"""Virtual Cottage""","""топ за свои деньги"""
109152307,204030,"""Fable - The Lost Chapters""","""Fun fact: this game taught me …"
71576818,431960,"""Wallpaper Engine""","""最喜欢的就是 去浏览 好友（yiwan）所订阅的壁纸"""


In [10]:
def extract_json_block(text):
    matches = re.findall(
        r'{\s*"visual_perception"\s*:\s*(".*?"|null),\s*"audio_perception"\s*:\s*(".*?"|null)\s*}',
        text,
        re.DOTALL
    )
    if matches:
        try:
            last_json = '{ "visual_perception": %s, "audio_perception": %s }' % matches[-1]
            return json.loads(last_json)
        except json.JSONDecodeError:
            pass
    return {"visual_perception": None, "audio_perception": None}

In [11]:
results = []

for row in sample_reviews_df.iter_rows(named=True):
    review_id = row["recommendationid"]
    game_title = row["game"]
    review_text = row["review"]

    prompt = make_prompt(review_text)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.2,
            top_p=1.0,
            do_sample=True
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    parsed = extract_json_block(decoded)

    results.append({
        "recommendationid": review_id,
        "game": game_title,
        "review": review_text,
        "visual_perception": parsed["visual_perception"],
        "audio_perception": parsed["audio_perception"]
    })

In [12]:
pl.DataFrame(results)

recommendationid,game,review,visual_perception,audio_perception
i64,str,str,str,str
93432934,"""Need for Speed™ Hot Pursuit Re…","""Just a graphical upgrade with …","""slightly worse looking graphic…",null
16366885,"""Mercenary Kings""","""ข้อดี - สนุกมาก - คราฟปืน,มีด…",null,null
99426611,"""Tom Clancy's Rainbow Six Siege""",""":) 8) me gusta bastante """,null,null
96427458,"""UNBEATABLE [white label]""","""Ohhh my!!!!! Who ever is readi…","""incredible one of a kind art s…","""Sick songs (love the drums)"""
80204144,"""Fallout: New Vegas PCR""","""9 гулей в ракете из 10""",null,null
…,…,…,…,…
127110915,"""Virtual Cottage""","""топ за свои деньги""",null,null
109152307,"""Fable - The Lost Chapters""","""Fun fact: this game taught me …",null,null
71576818,"""Wallpaper Engine""","""最喜欢的就是 去浏览 好友（yiwan）所订阅的壁纸""",null,null


In [ ]:
df.write_csv("../data/user_reviews/processed/all_reviews_processed.csv")